In [1]:
import numpy as np
import datetime, os, cv2, pickle, sys


from tifffile import imread

In [2]:
def write_pickle(path, obj):
    with open(path, "wb") as f:
        pickle.dump(obj, f)
        
def read_pickle(path):
    if not os.path.isfile(path):
        print(f"path did not exist: {path}")
        return None
    with open(path, "rb") as f:
        obj = pickle.load(f)
    return obj

In [3]:
base_path = '/mnt/DataRAID/melismail/PDAC/data'
preprocessing_path ='Preprocessing_images'
img_path = 'images/pdac_he_warped/images'
mask_path = 'images/pdac_he_warped/masks'
sliced_path = "sliced"
filtered_path = "filtered"
scaled_path = "scaled"
tile_path = "tiles"
dim = 512
dim_path = f"{dim}x{dim}"

Preprocessing functions

Slice function - slices the raw input image into desired shape, therefore change tuple in target_dimension

In [4]:
def img_Sliced(img, target_dimension: tuple):
    dim = img.shape
    #print(dim)
    #print(dim[0]//target_dimension[0], dim[1]//target_dimension[1])
    imgs_sliced = []
    #for (x_offset, y_offset) in [(0, 0), (int(target_dimension[0]/2), 0), (0, int(target_dimension[1]/2)), (int(target_dimension[0]/2), int(target_dimension[1]/2))]:
    #for (x_offset, y_offset) in [(0, 0)]:
        #logging.info(f"file: {file}, x_offset: {x_offset}, y_offset: {y_offset}")
    for x in range(dim[0]//target_dimension[0]+1):
        for y in range(dim[1]//target_dimension[1]+1):
            
            
            #print("Obere Ecke", x, y, dim[0]//target_dimension[0], dim[1]//target_dimension[1])
            if x == (dim[0]//target_dimension[0]):
                if y == (dim[1]//target_dimension[1]):
                    imgs_sliced.append(img[x*target_dimension[0]: dim[0], y*target_dimension[1]: dim[1]])
                   #print("Ecke", x*target_dimension[0], dim[0], y*target_dimension[1], dim[1]##)
                    continue
                imgs_sliced.append(img[x*target_dimension[0]: dim[0], y*target_dimension[1]: (y+1)*target_dimension[1]])
                #peinr(["Unten",x*target_dimension[0], dim[0], y*target_dimension[1], (y+1)*target_dimension[1]])
            elif y == (dim[1]//target_dimension[1]):
                imgs_sliced.append(img[x*target_dimension[0]: (x+1)*target_dimension[0], y*target_dimension[1]: dim[1]])
                #print("Rechts", x*target_dimension[0], (x+1)*target_dimension[0], y*target_dimension[1], dim[1])
            else:
                imgs_sliced.append(img[x*target_dimension[0]: (x+1)*target_dimension[0], y*target_dimension[1]: (y+1)*target_dimension[1]])
                #print("Mitte", x*target_dimension[0], (x+1)*target_dimension[0], y*target_dimension[1], (y+1)*target_dimension[1])
    
    return imgs_sliced

In [5]:
sliced_img_dict = read_pickle(path=os.path.join(base_path, preprocessing_path, sliced_path, dim_path, f"{sliced_path}_{dim_path}_img_dict.pkl"))
if sliced_img_dict is None:

    sliced_img_dict = {(file.split('_')[0], idx):  sliced_img
                       for idx, file in enumerate([f
                                                   for f in os.listdir(os.path.join(base_path, img_path)) 
                                                   if os.path.isfile(os.path.join(base_path, img_path, f)) and ".tif" in f])
                       for idx, sliced_img in enumerate(img_Sliced(img=cv2.imread(os.path.join(base_path, img_path, file), cv2.COLOR_BGR2RGB), 
                                                                   target_dimension=(dim, dim)
                                                                  )
                                                       )
                      }
    write_pickle(path=os.path.join(base_path, preprocessing_path, sliced_path, dim_path, f"{sliced_path}_{dim_path}_img_dict.pkl"), obj=sliced_img_dict)
    print(f"Count of sliced IMAGES with dimensions {dim}:  {len(sliced_img_dict.items())}")
else:
    print("Read from Disk")

SyntaxError: invalid syntax (3578697584.py, line 14)

In [ ]:
sliced_mask_dict = read_pickle(path=os.path.join(base_path, preprocessing_path, sliced_path, dim_path, f"{sliced_path}_{dim_path}_mask_dict.pkl"))
if sliced_mask_dict is None:
    
    sliced_mask_dict = {(file.split('_')[0], idx):  sliced_img
                       for idx, file in enumerate([f
                                                   for f in os.listdir(os.path.join(base_path, mask_path)) 
                                                   if os.path.isfile(os.path.join(base_path, mask_path, f)) and ".tif" in f])
                       for idx, sliced_img in enumerate(img_Sliced(img=cv2.imread(os.path.join(base_path, mask_path, file), cv2.COLOR_BGR2RGB), 
                                                                   target_dimension=(dim, dim)
                                                                  )
                                                       )
                      }

    write_pickle(path=os.path.join(base_path, preprocessing_path, sliced_path, dim_path, f"{sliced_path}_{dim_path}_mask_dict.pkl"), obj=sliced_mask_dict)
    print(f"Count of sliced MASK with dimensions {dim}: {len(sliced_mask_dict.items())}")
else: 
    print("Read from Disk")

Filter functions - filters out the tiles that don't have the desired target_dimension

In [ ]:
def filter_img(img, thresholdWidth:int, thresholdHeight:int):
    try:
        height, width, channels = img.shape
    except Exception as e:
        height, width = img.shape

    # if only height is smaller the thresholdHeight
    if (height < thresholdHeight and width >= thresholdWidth):
        return False      
    # if only width is smaller the thresholdWidth
    elif (height >= thresholdHeight and width < thresholdWidth): 
        return False
    # if both the parameters are smaller
    # the threshold attributes
    elif (height < thresholdHeight and width < thresholdWidth):
        return False
        
    return True

In [ ]:
filtered_img_dict = read_pickle(path=os.path.join(base_path, preprocessing_path, filtered_path, dim_path, f"{filtered_path}_{dim_path}_img_dict.pkl"))
if filtered_img_dict is None:
        
        filtered_img_dict = {k: v for k, v in sliced_img_dict.items() if filter_img(img=v, thresholdWidth=dim, thresholdHeight=dim)}
       
    write_pickle(path=os.path.join(base_path, preprocessing_path, filtered_path, dim_path, f"{filtered_path}_{dim_path}_img_dict.pkl"), obj=filtered_img_dict)
    print(f"Count of FILTERED IMAGES with dimensions {dim}:  {len(filtered_img_dict.items())}")
else:
    print("Read from Disk")

In [ ]:
filtered_mask_dict = read_pickle(path=os.path.join(base_path, preprocessing_path, filtered_path, dim_path, f"{filtered_path}_{dim_path}_mask_dict.pkl"))
if filtered_mask_dict is None:
    filtered_mask_dict = {k: v for k, v in sliced_mask_dict.items() if filter_img(img=v, thresholdWidth=dim, thresholdHeight=dim)}
    

    write_pickle(path=os.path.join(base_path, preprocessing_path, filtered_path, dim_path, f"{filtered_path}_{dim_path}_mask_dict.pkl"), obj=filtered_mask_dict)
    print(f"Count of FILTERED MASKS with dimensions {dim}:  {len(filtered_img_dict.items())}")
else:
    print("Read from Disk")

Preprocessing function for StarDist - slices the filtered tiles into 4 pieces and upscales them again to the desired dimension

In [ ]:
def second_slice(img):
    try:
        h, w, channels = img.shape
    except Exception as e:
        h, w = img.shape
    vertical = w//2
    horizontal = h//2

  # this will be the first column
    left_top = img[:horizontal , :vertical]
    left_bottom = img[horizontal: , :vertical]
    
  # [:,:half] means all the rows and
  # all the columns upto index half
  # this will be the second column
    right_top = img[:horizontal, vertical:]
    right_bottom = img[horizontal:, vertical:]
  # this is horizontal division
    new_slices = [left_top, left_bottom, right_top, right_bottom]

    return new_slices


def upscale(img, scale_factor = 2):
    width = int(img.shape[1] * scale_factor)
    height = int(img.shape[0] * scale_factor)
    dim = (width, height)
    
    return cv2.resize(img, dim, interpolation = cv2.INTER_AREA)


def upscale_wrapper(img):
    img_list = second_slice(img)
    img_list = [upscale(i, scale_factor = 2) for i in img_list]
    
    return img_list

In [ ]:
img_dict_scaled = read_pickle(path=os.path.join(base_path, preprocessing_path, scaled_path, dim_path, f"{scaled_path}_{dim_path}_img_dict.pkl"))
if img_dict_scaled is None:
    img_dict_scaled = {(k,idx): scaled_img 
                        for k, v in filtered_img_dict.items() 
                        for idx, scaled_img in enumerate(upscale_wrapper(v))}


    write_pickle(path=os.path.join(base_path, preprocessing_path, scaled_path, dim_path, f"{scaled_path}_{dim_path}_img_dict.pkl"), obj=img_dict_scaled)
    print(f"Count of SCALED IMAGES with dimensions {dim}:  {len(img_dict_scaled.items())}")
else:
    print("Read from Disk")

In [ ]:
mask_dict_scaled = read_pickle(path=os.path.join(base_path, preprocessing_path, scaled_path, dim_path, f"{scaled_path}_{dim_path}_mask_dict.pkl"))
if mask_dict_scaled is None:
    mask_dict_scaled = {(k,idx): scaled_img
                        for k,v in filtered_mask_dict.items()
                        for idx, scaled_img in enumerate(upscale_wrapper(v))}

    write_pickle(path=os.path.join(base_path, preprocessing_path, scaled_path, dim_path, f"{scaled_path}_{dim_path}_mask_dict.pkl"), obj=mask_dict_scaled)
    print(f"Count of SCALED MASKS with dimensions {dim}:  {len(mask_dict_scaled.items())}")
else:
    print("Read from Disk")

In [ ]:
def plot_img(target_dir: str, img_dict: dict, identifier: tuple):
        cv2.imwrite(os.path.join(target_dir, f"{identifier[0]}_{identifier[1]}.jpg", v)

In [ ]:
plot_img(target_dir = os.path.join(base_path, preprocessing_path, tile_path), img_dict=filtered_img_dict, identifier=("",))